In [1]:
import os
import cv2
import tensorflow as tf
import joblib
import numpy as np
from ultralytics import YOLO
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input

# Charger le modèle SVM
svm_model_path = 'B:\\computerVISIONFINALE\\best_svm_resnet.pkl'  # Remplacez par le chemin de votre modèle SVM
svm_model = joblib.load(svm_model_path)

# Charger le modèle YOLOv8
model = YOLO("yolov8n.pt")  # Utilisez yolov8n.pt pour un modèle léger ou un autre modèle selon vos besoins

# Liste des déchets spécifiques à détecter
waste_objects = [
    'plastic',  # Plastique
    'paper',    # Papier
    'fruit',    # Fruits
    'bottle',   # Bouteille
    'glass',    # Verre
    'cardboard',# Carton
    'can',      # Canette
    'metal',    
    'orange',   # Orange
    'banana'    # Banane
]

# Dictionnaire des classes YOLO correspondantes (exemple basé sur COCO ou classes personnalisées)
yolo_class_dict = {
    'plastic': 56,    # Classe pour le plastique (exemple d'ID COCO)
    'paper': 67,      # Classe pour le papier
    'fruit': 52,      # Classe pour les fruits (exemple)
    'bottle': 39,     # Classe pour la bouteille
    'glass': 77,      # Classe pour le verre
    'cardboard': 56,  # Classe pour le carton
    'can': 39,        # Classe pour la canette
    'metal': 40,      # Classe pour le métal
    'orange': 47,     # Classe pour l'orange (à vérifier)
    'banana': 43      # Classe pour la banane (à vérifier)
}

# Charger le modèle ResNet50 pré-entraîné de TensorFlow
resnet_model = ResNet50(weights='imagenet', include_top=False, pooling='avg')  # Utiliser le modèle sans la couche finale

# Fonction pour appliquer YOLOv8 sur une image et détecter les objets spécifiques
def detect_objects_yolov8(image_path):
    img = cv2.imread(image_path)
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    # Utiliser YOLOv8 pour détecter les objets dans l'image
    results = model.predict(image_path)  # Utiliser predict au lieu de img_path

    # Extraire les résultats des détections
    boxes = results[0].boxes  # Boîtes englobantes des objets détectés
    class_ids = boxes.cls  # Classes des objets détectés
    confidences = boxes.conf  # Confiances des détections

    return boxes, class_ids, confidences, img_rgb

# Fonction pour extraire les caractéristiques d'un objet pour la classification SVM
def extract_features_for_svm(object_image):
    # Convertir l'image de BGR à RGB si nécessaire
    if object_image.shape[2] == 3:  # Vérifiez si l'image a 3 canaux (RGB)
        object_image = cv2.cvtColor(object_image, cv2.COLOR_BGR2RGB)

    # Redimensionner l'image pour qu'elle corresponde à la taille attendue par ResNet50
    object_image = cv2.resize(object_image, (224, 224))  # 224x224 est la taille d'entrée attendue
    img_array = image.img_to_array(object_image)  # Convertir l'image en tableau numpy
    img_array = np.expand_dims(img_array, axis=0)  # Ajouter une dimension pour le batch
    img_array = preprocess_input(img_array)  # Appliquer la prétraitement spécifique à ResNet50

    # Passer l'image à travers le modèle ResNet50 pour obtenir les caractéristiques
    features = resnet_model.predict(img_array)  # Extraire les caractéristiques
    return features.flatten()  # Aplatir la sortie et la convertir en numpy

# Fonction principale pour traiter les images et prédire les classes avec YOLOv8 et SVM
def process_images(image_dir, output_dir):
    # Créer le dossier de sortie si nécessaire
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    image_count = 0
    for filename in os.listdir(image_dir):
        if filename.endswith('.jpg') or filename.endswith('.png'):
            # Charger l'image
            image_path = os.path.join(image_dir, filename)
            boxes, class_ids, confidences, img_rgb = detect_objects_yolov8(image_path)

            # Filtrer les objets selon la liste de déchets
            detected_objects = []
            for i, class_id in enumerate(class_ids):
                # Vérifier si l'objet détecté est dans notre liste
                for obj in waste_objects:
                    if int(class_id) == yolo_class_dict.get(obj):
                        # Extraire les coordonnées de la boîte englobante
                        x1, y1, x2, y2 = boxes[i].xyxy[0].cpu().numpy()  # Convertir en liste pour un déballage correct

                        # Extraire l'objet de l'image
                        object_image = img_rgb[int(y1):int(y2), int(x1):int(x2)]

                        # Extraire les caractéristiques de l'objet
                        features = extract_features_for_svm(object_image)

                        # Classifier l'objet avec le modèle SVM
                        prediction = svm_model.predict([features])

                        # Afficher le résultat de la classification
                        if prediction == 0:
                            label = "Recyclable"
                        else:
                            label = "Organic"
                        
                        detected_objects.append((obj, label))

                        # Choisir la couleur de la boîte en fonction de l'étiquette
                        if label == "Organic":
                            color = (255, 0, 0)  # Bleu pour "Organic"
                        else:
                            color = (0, 255, 0)  # Vert pour "Recyclable"

                        # Dessiner une boîte englobante et un label sur l'image
                        cv2.rectangle(img_rgb, (int(x1), int(y1)), (int(x2), int(y2)), color, 2)
                        cv2.putText(img_rgb, f"{label}", (int(x1), int(y1) - 10),
                                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

            # Si des objets ont été détectés, afficher l'image et les résultats
            if detected_objects:
                print(f"Image: {filename}")
                for obj, label in detected_objects:
                    print(f"Objet détecté: {obj} - Classe: {label}")

                # Enregistrer l'image annotée dans le dossier de sortie
                output_path = os.path.join(output_dir, f"processed_{filename}")
                cv2.imwrite(output_path, cv2.cvtColor(img_rgb, cv2.COLOR_RGB2BGR))

                # Incrémenter le compteur d'images traitées
                image_count += 1


# Répertoire d'images à traiter
image_dir = r'B:\\computerVISIONFINALE\\images'  # Chemin vers le répertoire contenant les images

# Dossier de sortie pour les images annotées
output_dir = r'B:\\computerVISIONFINALE\\output_images'  # Remplacez par le chemin de votre dossier de sortie

# Appliquer la détection et la classification
process_images(image_dir, output_dir)



image 1/1 B:\computerVISIONFINALE\images\images.jpg: 384x640 1 cup, 1 spoon, 4 oranges, 104.5ms
Speed: 15.9ms preprocess, 104.5ms inference, 11.8ms postprocess per image at shape (1, 3, 384, 640)

image 1/1 B:\computerVISIONFINALE\images\image_1.jpg: 640x480 1 person, 1 bottle, 1 pizza, 1 dining table, 1 vase, 63.5ms
Speed: 2.0ms preprocess, 63.5ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 480)
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step
Image: image_1.jpg
Objet détecté: bottle - Classe: Recyclable
Objet détecté: can - Classe: Recyclable

image 1/1 B:\computerVISIONFINALE\images\image_10.jpg: 640x480 1 bottle, 58.3ms
Speed: 0.6ms preprocess, 58.3ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 480)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
Image: image_10.jpg
Objet détecté: bottle - Classe: Recyclable
Objet détecté: can - Classe: Recyclable

image 1/1 B:\computerVISIONFINALE\images\image_11

In [2]:
import joblib
import pickle

# Fonction pour sauvegarder le modèle YOLO et SVM ensemble
def save_final_model(yolo_model_path, svm_model_path, final_model_path):
    final_model = {
        "yolo_model_path": yolo_model_path,
        "svm_model_path": svm_model_path,
        "resnet_model": resnet_model
    }
    with open(final_model_path, 'wb') as file:
        pickle.dump(final_model, file)
    print(f"Final model saved to {final_model_path}")

# Sauvegarder le modèle final
final_model_path = r'B:\\computerVISIONFINALE\\final_model.pkl'  # Chemin où le modèle sera sauvegardé
save_final_model("yolov8n.pt", svm_model_path, final_model_path)


Final model saved to B:\\computerVISIONFINALE\\final_model.pkl
